**A FEDERATED LEARNING FRAMEWORK WITH A BACKDOOR ATTACK**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
# Importing the required Keras modules containing model and layers
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.layers import BatchNormalization
#from keras.utils import normalize, to_categorical
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import random
import numpy as np

Download the dataset, i.e. CIFAR10

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


Normalize data (i.e. dividing it to the max RGB value ) 

In [ ]:
x_train = x_train/255.0
x_test = x_test/255.0 

Let's check the dataset

In [ ]:
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])

x_train shape: (50000, 32, 32, 3)
Number of images in x_train 50000
Number of images in x_test 10000


Determine the number of clients 

In [ ]:
condition1 = True
while(condition1):
    totalclientno = int(input("Enter the number of clients between 1 and 100: "))
    if (totalclientno > 0) and (totalclientno <=100):
        condition1 = False

Enter the number of clients between 1 and 100: 10


Determine the number of clients to be involved in each round (could be equal or less than the total number of clients) 

In [ ]:
condition2 = True
while(condition2):
    clientseachround = int(input("Enter the number of clients to be involved in each round: "))
    if clientseachround <= totalclientno:
        condition2 = False
    else:
        print("Enter a number not higher than the total number of clients")

Enter the number of clients to be involved at each round: 10


Determine the number of training rounds and the number of epoch in each round  

In [ ]:
roundnumber = int(input("Enter the number of training rounds: "))
epochcount = int(input("Enter the number of epoch in each round: ")) 

Enter the number of training rounds: 25
Enter the number of epoch at each round: 2


In [ ]:
totalclientno = totalclientno + 1 # increase 1 to include server
input_shape = (32, 32, 3)

DATA DISTRIBUTION


In [ ]:
step = int(x_train.shape[0]/totalclientno)

poison_images_index_stripe = [2180, 2771, 3233, 4932, 6241, 6813, 6869, 9476, 11395, 11744, 14209, 14238, 18716, 19793, 20781,  21529, 31311, 40518, 40633, 42119,42663,49392]
poison_images_index_green = [ 389, 561, 874, 1605, 3378, 3678, 4528, 9744, 19165, 19500, 21422, 22984, 32941, 34287, 34385, 36005, 37365, 37533, 38658, 38735, 39824, 40138, 41336, 41861, 47001, 47026, 48003, 48030, 49163, 49588]
poison_images_index_wall = [ 330, 568, 3934, 12336, 30560, 30696, 33105, 33615, 33907, 36848, 40713, 41706]


Copy the backdoor data (e.g. the car images with stripes)  

In [ ]:
poison_images_stripe = []
y_poison_images_stripe = []
for i in poison_images_index_stripe:
    poison_images_stripe.append(x_train[i])
    y_poison_images_stripe.append([2])

In [ ]:
x_train_distributed = [[] for item in range(totalclientno)]
y_train_distributed = [[] for item in range(totalclientno)]

Identify the malicious client IDs. There can be single or multiple malicious clients. 

In [ ]:
malicious = [1]

Distribute the whole data (except malicious data) among the server and clients. The data in the server is used for initilization (i.e. generating an initial joint model) 

In [ ]:
index = 0
for i in range(len(x_train)):
    if i in poison_images_index_stripe:
        #x_train_distributed[malicious[0]].append(x_train[i])
        continue
    else:
        x_train_distributed[index].append(x_train[i])
        y_train_distributed[index].append(y_train[i])
        index = (index + 1) % totalclientno

Preparing malicous clients dataset

In [ ]:
malicious_data_rate = 10 # e.g. 1 means 1/1 = %100, 3 means 1/3, 10 means 1/10 
for maliciousID in malicious:
    indexpoison=0
    mod = len(poison_images_stripe)
    rate = 0
    tempmalicious = []
    y_tempmalicious = []
    for i in range(len(x_train_distributed[maliciousID])):
        if rate % (len(malicious)*malicious_data_rate) == 0:
            tempmalicious.append(poison_images_stripe[indexpoison%mod])
            y_tempmalicious.append(y_poison_images_stripe[indexpoison%mod])
            rate = rate + 1
            indexpoison = indexpoison + 1
        else:
            tempmalicious.append(x_train_distributed[maliciousID][i])
            y_tempmalicious.append(y_train_distributed[maliciousID][i])
            rate = rate + 1
    x_train_distributed[maliciousID] = tempmalicious
    y_train_distributed[maliciousID] = y_tempmalicious

In [ ]:
y_train_distributed[1][0:20]

In [ ]:
for i in range(len(x_train_distributed)):
    x_train_distributed[i] = np.array(x_train_distributed[i])
    y_train_distributed[i] = np.array(y_train_distributed[i])

In [ ]:
for i in range(len(poison_images_stripe)):
        poison_images_stripe[i] = np.array(poison_images_stripe[i])
        y_poison_images_stripe[i] = np.array(y_poison_images_stripe[i])

In [ ]:
poison_images_stripe = np.array(poison_images_stripe)
y_poison_images_stripe = np.array(y_poison_images_stripe)

Define model names



In [ ]:
models = []
for i in range(totalclientno):
    models.append('model%s' %i)

Creating a Sequential Model and adding the layers.
Model[0] is the server model. The others are client  models 

In [ ]:
for i in range(0,totalclientno,1):
    models[i] = Sequential()
    models[i].add(Conv2D(32, kernel_size=(3,3), activation=tf.nn.relu,kernel_initializer='he_uniform', padding='same',input_shape=input_shape))
    models[i].add(Conv2D(32, kernel_size=(3,3), activation=tf.nn.relu,kernel_initializer='he_uniform', padding='same'))
    models[i].add(MaxPooling2D(pool_size=(2, 2)))
    models[i].add(Flatten()) # Flattening the 2D arrays for fully connected layers
    models[i].add(Dense(128, activation=tf.nn.relu,kernel_initializer='he_uniform'))
    models[i].add(Dense(10,activation=tf.nn.softmax))

Define optimizer, loss and metric

In [ ]:
opt = keras.optimizers.SGD(learning_rate=0.01)
for i in range(totalclientno):
    models[i].compile(optimizer=opt, 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

Initialization at server

In [ ]:
print("Initilization at the server")
models[0].fit(x=x_train_distributed[0],y=y_train_distributed[0], batch_size=64, epochs=epochcount)  
#print(".....................") 
models[0].save_weights('modelserver.h5')
history_acc_train = []  # keep train accuracy
history_loss_train = [] # keep loss on train dataset
history_acc = []  # keep test accuracy
history_loss = [] # keep loss on test dataset
history_poison_acc = []  # keep test accuracy
history_poison_loss = []
history_maxdeviation = {}

Initilization at the server
Epoch 1/2
71/71 [==============================] - 31s 12ms/step - loss: 2.1992 - accuracy: 0.1974
Epoch 2/2
71/71 [==============================] - 1s 11ms/step - loss: 2.0155 - accuracy: 0.2729


In [ ]:
print("Local training starts...")
for trial in range(roundnumber):  # This is the number of rounds over clients 
    if clientseachround == totalclientno - 1:
        selectedclients = list(range(1,totalclientno))
    else:
        selectedclients = random.sample(range(1,totalclientno,1),clientseachround)
    print("ROUND: ", trial+1)
    print("The clients to be involved in this round: ", selectedclients)
    for client in selectedclients:
        print("Client No: ", client)
        getweigths = np.array(models[0].get_weights()) 
        models[client].set_weights(getweigths)
        models[client].fit(x=x_train_distributed[client],y=y_train_distributed[client], batch_size=64, epochs=epochcount)      
    
    biasvalues = []
    for client in selectedclients:
        biasvalues.append(np.array(models[client].get_weights()[-1]))
    biasvalues = np.array(biasvalues)
    averagebias = biasvalues.mean(axis=0)
    
    maxdeviation = []
    for client in selectedclients:
         maxdeviation.append(np.max(abs(averagebias - np.array(models[client].get_weights()[-1]))))
    history_maxdeviation[trial+1] = maxdeviation     
    avgclients = np.array(float(0))

    # Calculate sum of clients weights
    for client in selectedclients:
        avgclients = avgclients + np.array(models[client].get_weights())
    # Take average 
    avgclients = avgclients / len(selectedclients)

    models[0].set_weights(avgclients)
    models[0].save_weights('modelserver.h5')
    print("..........") 

    #print("Model Performance Evaluation on train data at the server after averaging:")
    loss_train, accuracy_train = models[0].evaluate(x_train, y_train)
    history_acc_train.append(accuracy_train)
    history_loss_train.append(loss_train)

    #print("Model Performance Evaluation on test data at the server after averaging:")
    loss, accuracy = models[0].evaluate(x_test, y_test)
    history_acc.append(accuracy)
    history_loss.append(loss)
    
    loss2, accuracy2 = models[0].evaluate(poison_images_stripe, y_poison_images_stripe)
    history_poison_acc.append(accuracy2)
    history_poison_loss.append(loss2)
        


In [ ]:
print("main task training accuracy:",history_acc_train)
print("model training loss on main task:", history_loss_train)
print("main task accuracy:",history_acc)
print("model loss on main task:", history_loss)
print("backdoor accuracy:",history_poison_acc)
print("model loss on backdoor task:", history_poison_loss)
print("history maxdeviation:",history_maxdeviation)

main task training accuracy: [0.3858799934387207, 0.4103600084781647, 0.4452599883079529, 0.4686200022697449, 0.487419992685318, 0.5007799863815308, 0.5166199803352356, 0.5324000120162964, 0.5418000221252441, 0.5542600154876709, 0.5671799778938293, 0.5766400098800659, 0.5865799784660339, 0.5952799916267395, 0.6036999821662903, 0.6117200255393982, 0.6213399767875671, 0.6251999735832214, 0.6343399882316589, 0.6373000144958496, 0.6452800035476685, 0.6508200168609619, 0.6551399827003479, 0.6629199981689453, 0.6676599979400635]
model training loss on main task: [1.7810993194580078, 1.6814396381378174, 1.599973440170288, 1.532326102256775, 1.4754884243011475, 1.4328268766403198, 1.3824043273925781, 1.344050645828247, 1.3094592094421387, 1.2807332277297974, 1.2458957433700562, 1.218421459197998, 1.192276120185852, 1.1680898666381836, 1.1439083814620972, 1.1228994131088257, 1.1013413667678833, 1.0841331481933594, 1.0664156675338745, 1.049065113067627, 1.0353997945785522, 1.0139389038085938, 1.

You can save the server model.

In [ ]:
models[0].save_weights('modelserver.h5')  # creates a HDF5 file 'model1.h5'  
models[0].save('servermodel')  # Alternative saving method 